# Homework 3

In this notebook, you will explore kernel ridge regression and kernel SVM. We first present kernel ridge regression on a housing dataset to showcase the ideas in Question 4 on the theoretical portion of the homework. Next, we start our exploration into kernel SVM with a two-dimensional example on the spiral data and then build a simple but powerful sentiment classifier on tweets to airlines, a topic we may have more sympathy for as inclement weather hits us here in Chicago...

## Preparation

In [ ]:
!wget -O $PWD/utils.py https://www.dropbox.com/scl/fi/9od7hx1y2q0jf7sncsk1w/utils.py?rlkey=c3gy7aiphk8ycfb9qjgsm5a06&dl=1

In [ ]:
import os
import numpy as np
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn import svm
import matplotlib.pyplot as plt
import utils

## Kernel Ridge Regression

In the Question 3 of the theoretical homework, we studied kernel ridge regression. In this part, we will address the practical considerations for solving the kernel ridge regression problem.

We consider the [California housing dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset), which consists of 8 numeric features, including house age, number of bedrooms, and location, and has target median house value. It contains over 20 thousand samples. We will use 3000 for now, but feel free to use more to see what happens! First, let us load it from `sklearn`'s set of datasets.

In [ ]:
cal  = sklearn.datasets.fetch_california_housing()

In [ ]:
from utils import TrainAndTestData

seed = 0
np.random.seed(seed)

m = 3000
perm = np.random.permutation(len(cal.target))
train_i = perm[:m]
test_i = perm[m:]
train_X = cal.data[train_i,:]
train_y = cal.target[train_i]
test_X = cal.data[test_i,:]
test_y = cal.target[test_i]

housing_data = TrainAndTestData(train_X, train_y, test_X, test_y)

Let use a Gaussian (RBF) kernel. Recall that the definition of this kernel is:

$$
    K_{RBF}(x_i, x_j) = \exp \left( -\beta \left(  \langle x_i, x_i \rangle + \langle x_j, x_j \rangle - 2\langle x_i, x_j \rangle  \right)    \right)
$$

In [ ]:
def RBF_kernel(beta = 1):
    def RBF_kernel_beta(x1,x2):
        return np.exp(- beta*(np.sum(x1*x1, 1)[:,np.newaxis] + np.sum(x2*x2, 1)-2*x1@x2.T ))
    return RBF_kernel_beta

*A quick note on closure: the above kernel construction function employs a Python concept known as `closure` which provides us the following functionality. Every kernel function should have the same signature: given two data points as input, output a real number. However, the function may be dependent on some parameter that we cannot hard code. Thus, the outer function constructs a kernel function with the desired signature while fixing a value for the parameter. This becomes very useful when we may need to pass around a kernel function but always for the same parameter value*

To use this function, we first pick some value of `beta` and instantiate: `RBF_kernel(beta=myvalue)`. This is itself a function and can now take in matrices `x1, x1`. That is:

In [ ]:
RBF_kernel(beta = 1)(housing_data.X_train[:10], housing_data.X_train[:10])

### [Task 1] Looking for a great $\beta$ value
Look at the values of the kernel for that setting of $\beta$. What do you notice about the values? Generally, what values is the Gaussian kernel distributed between (consider limiting cases: large distance / exponent, small distance exponent). What part of that range does the above lie in? How can you improve this? Find a good setting for $\beta$ by considering these questions.

In [ ]:
#### TASK 1 CODE
RBF_kernel(beta = ???)(housing_data.X_train[:10], housing_data.X_train[:10])
#### TASK 1 CODE

Armed with a good kernel to represent our features well, we move on to the learning algorithm.

### [Task 2] Implementing Kernel Ridge Regression
Now, let us implement the kernel ridge regression solution. In 3(b), you formulated the solution to the kernel ridge regression as computing the least squares solution to some expression. Finish the function below using proper numeric python syntax. Also compute the prediction given the kernel, the validation points, the predictor, and the training points (`predict_kernel_ridge`).

In [ ]:
def train_kernel_ridge(kernel, lmbd, x, y):
    from numpy.linalg import lstsq
    K = kernel(x,x)
    #### TASK 2 CODE
    least_squares_soln =
    #### TASK 2 CODE
    return least_squares_soln

In [ ]:
def predict_kernel_ridge(kernel, x, alpha, train_x):
    #### TASK 2 CODE
    #### TASK 2 CODE

Given this, load and process the data into the Gram matrix, compute the KRR solution for this data for some fixed regularization parameter $\lambda$, and predict the answers on a validation set.

**Check yourself**: How long does the `train_kernel_ridge` function take to run? Are you inverting a matrix?

Great. You now are able to compute the KRR solution for data. Let's compute some baselines to know what we're trying to beat. Compute the mean squared error for the following two (extremely simple) predictors:
* **null predictor**: output 0 for every data point
* **mean predictor**: output the mean of the training `y` for every data point

In [ ]:
def mean_squared_error(pred, y):
    return np.mean((pred-y)**2)

In [ ]:
#### TASK 2 CODE
## BASELINES

#### TASK 2 CODE

<span style="color: red">
<h4 style="font-weight: bold">[Answer Question 1]</h4>

Use cross-validation or validation to choose $\lambda$. Make sure you beat the baselines. What values of $\beta, \lambda$ give good performance? <br>

<h4 style="font-weight: bold">---------------------</h4>

<span style="color: blue">
Answer:
</span>

<h4 style="font-weight: bold">---------------------</h4>
</span>

In [ ]:
#### CODE
#### CODE

## 2D kernel SVM

In this part, we revisit the spiral dataset. We will walk through solving this problem using the radial basis function kernels. Your job will be to understand how well the predictor works. After this, we encourage you to explore trying different kernels and parameter settings to see how they perform. We will use the [implementation of SVM in `scikit-learn`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

In [ ]:
np.random.seed(0)

LABELS = [-1, 1]
SP_THETA_SIGMA = 0.3
SP_R_SIGMA = 0.05
NOISE_LEVEL = 0.2

m = 1000
Xsp, ysp = utils.generate_spiral_data(m, noise_level=NOISE_LEVEL, theta_sigma=SP_THETA_SIGMA, r_sigma=SP_R_SIGMA)

train_test_ratio = 0.8
Xsp_train, ysp_train, Xsp_test, ysp_test = utils.create_split(Xsp, ysp, train_test_ratio)

spirals = TrainAndTestData(Xsp_train, ysp_train, Xsp_test, ysp_test)

See how RBF kernel (sigma) compares to the $k$-Nearest Neighbor predictor and decision tree predictors from HW3.

In [ ]:
# for now, kernel = one of ['linear', 'poly', 'rbf']
# the relevant parameters are degree, gamma -- see documentation for details of how to use them
svm_clf = svm.SVC(kernel='rbf', gamma=10, C=5)
svm_clf = svm_clf.fit(Xsp_train, ysp_train)
utils.plot_decision_boundary(svm_clf, Xsp_train, ysp_train, Xsp_test, ysp_test)

spirals.print_errors(svm_clf)

### [Task 3] Applying SVM classifiers to the 2D spiral data
For the classifier you trained, find and write down the value of the SVM object, training loss, and training error, margin violations, and the number of support vectors.

In [ ]:
#### TASK 3 CODE
#### TASK 3 CODE

<span style="color: red">
<h4 style="font-weight: bold">---------------------</h4>

<span style="color: blue">
Answer:
</span>

<h4 style="font-weight: bold">---------------------</h4>
</span>

Explore different kernels, parameter values, and regularization parameters to see how these factors affect the learned classifier. Spend some time exploring here to understand the influence of these factors, since in the next part, you will be using the same techniques, but this time on data that lies in much higher than 2D, so plotting to analyze what you get is not really an option.

## Sentiment Analysis Using Kernel SVM

Next, we will develop a sentiment classifier using kernel SVM. We will work with a real-world dataset of [tweets to airlines](https://www.kaggle.com/crowdflower/twitter-airline-sentiment/version/2). Datasets that have been scraped from the internet (such as this one) are prone to many issues, whether we use them directly or with some filtering. See if you can think of a few potential issues, and feel free to discuss with TAs at office hours. Despite these issues, the dataset provides value in giving us short statements with strong sentiment that we will build a classifier over.

**Data pre-processing** The raw data, which you can access and study at the above link, contains 15 attributes, including `tweet_id` , `airline_sentiment`, `negative_reason`, `airline`, `text.` Of these, we are most interested in `airline_sentiment` and `text`. To that end, we have extracted these for you in the files `cleaned_tweets_train.tsv` and `cleaned_tweets_test.tsv`. A `tsv` file is a file where the different attributes are separated by tabs. The dataset identifies three different sentiments: `positive`, `neutral`, and `negative`. After extracting just the tweets and the sentiments, we shuffled all the tweets and saved the first 3/4 of them to the training file and the remaining 1/4 of them to the test file.

**Data loading** The `load_data` function we provide in `utils.py` allows filtering neutral (i.e., removing them from the data) by setting `filter_neutrals` flag to `True` or including them but counting them as "not positive" examples by setting `filter_neutrals` flag to `False`.

In [ ]:
!mkdir $PWD/data/
!wget -O $PWD/data/cleaned_tweets_train.tsv https://www.dropbox.com/scl/fi/tuh6nuhr0uqqj6tjxyd4p/cleaned_tweets_train.tsv?rlkey=xe71ynusj9fzbm1i7vrcq9gpp&dl=1
!wget -O $PWD/data/cleaned_tweets_test.tsv https://www.dropbox.com/scl/fi/q0e356xajsxjh40x7xwcr/cleaned_tweets_test.tsv?rlkey=hfenusk7tcvvi0xu85181dbcy&dl=1

### Kernel Functions

Recall that a kernel can be defined as $K_{ij} = \langle \phi(x_i), \phi(x_j) \rangle\, \in \mathbb{R}\,.$ However, the $x_i$ do not have to be real-valued, or numeric at all. Indeed, in this case, they are strings of length $k$ (tweets, in particular) in $\mathcal{D}^k$, where $\mathcal{D}$ is the dictionary of words. Then, we can decompose the kernel as:


$$
K_{ij} = \langle \phi(x_i), \phi(x_j) \rangle = \langle \tilde{\phi}(v(x_i)), \tilde{\phi}(v(x_j))\rangle\
$$


where $\phi = \tilde{\phi} \circ v\,,$ $\tilde{\phi} \, : \mathbb{R}^{d_1} \mapsto \mathbb{R}^{d_2}$ and $v \, : \mathcal{D}^k \mapsto \mathbb{R}^{d_1}\,.$

This decomposition allows us to separate the transformation into two parts. Now, we can choose both independently. Here are some suggestions for each:

* For $v(x_i)\,:$
    * **Bag-of-words**: for each word $w$ in the corpus, the corresponding component of the bag-of-words representation of $x_i$ is defined as the number of occurences of $w$ in $x_i$.
    * **Bi-gram**: for each pair of words that occur contiguously in the corpus, the corresponding component of the bi-gram representation of $x_i$ is the number of times that the bi-gram (two-word pattern) appears in $x_i$.
    * **Subsequence counts**: for each subsequence (of fixed size) in the corpus, the corresponding component of this representation of $x_i$ is the nubmer of times that the subsequence has appeared in the document $x_i.$ (A subsequence allows for skipping characters, whereas a substring is all continguous characters.)
    * ...others?



* For $\tilde{\phi}\,:$
    * Linear
    * Polynomial
    * Radial basis function
    * Weighted cosine similarity (word kernel in the pdf)
    * ...others?

Here, we implement several kernels with which you may experiment. **You should also try at least one combination that we have not implemented for you.**

**Check your understanding**:
* what is the dimension of a bag-of-words representation of a sentence? what about bigram?

In [ ]:
def BoW_inner(s1,s2):
    "returns inner product between bag-of-word feature vectors of the two input strings"
    from collections import Counter
    d1 = Counter(s1.split())
    return sum(d1[w] for w in s2.split())

<span style="color: red">
<h4 style="font-weight: bold">[Answer Question 2]</h4>

No coding in this one: Consider the BoW kernel constructed in `BoW_inner`. Suppose there are $D$ words in the corpus, and each sentence (document) has at most $k$ words.
* What is the time and space complexity of naively constructing the bag-of-words vector for each sentence and computing their inner product?
* What is the time and space complexity of the implementation in the code?
* What accounts for the difference? <br>

<h4 style="font-weight: bold">---------------------</h4>

<span style="color: blue">
Answer:
</span>

<h4 style="font-weight: bold">---------------------</h4>
</span>

### [Task 4] Implementing your own feature mapping function for text data

Think about your own understanding of sentences. What features do you use to understand them? **Hand design a feature mapping from sentence to numeric values that might help a kernel learn to classify sentiment.** To do this, you may wish to load the training data and inspect what positive and negative samples look like.

In [ ]:
X_train, y_train = utils.load_data(os.path.join(os.getcwd(),"data/cleaned_tweets_train.tsv"))

In [ ]:
print("---- positive samples ----")
for i in range(100):
    if y_train[i] == 1:
        print(X_train[i])
print("--------------------------")
print("---- negative samples ----")
for i in range(100):
    if y_train[i] == -1:
        print(X_train[i])
print("--------------------------")

In [ ]:
def my_feature_map(x1): ### You should rename this to be more descriptive about the feature mapping that you are using
    #### TASK 4 CODE
    #### TASK 4 CODE
    pass

In [ ]:
def my_inner_product(x1, x2):
    '''
    this function computes the inner product phi(x1)*phi(x2) for phi,
        the feature transform defined in the previous cell, i.e., my_feature_map
    implementing this as np.dot(my_feature_map(x1), my_feature_map(x2)) is not
        super-useful, as the runtime will be at least linear in the dimension of
        the feature map. Instead, implement this without ever using my_feature_map.
    '''
    #### TASK 4 CODE
    #### TASK 4 CODE
    pass

Next, we compute the Gram matrix from any kernel we have implemented (e.g., the one you just implemented or the bag-of-words example given in function `BoW_inner`. This is useful in computing $\tilde{\phi}$ for $\tilde{\phi}$ that can be vectorized.

In [ ]:
def gram_matrix(K):
    def gram_matrix_K(xs_1, xs_2):
        return np.array([[K(x1, x2) for x2 in xs_2] for x1 in xs_1])
    return gram_matrix_K

In [ ]:
def rbf_kernel_gram(inner, beta=1):
    """Gaussian RBF kernel.

    Returns a functoin gram(xs_1,xs_2) that calculate the (cross) gram matrix G[i,j]=K(xs_1[i,xs_2[j]])
    where K is the Gaussian RBF on the features phi, specified through the inner product in phi space."""
    def rbf_kernel_sigma_inner(xs_1,xs_2):
        return np.exp(-beta*(np.array([inner(x1, x1) for x1 in xs_1])[:, np.newaxis]
                             + np.array([inner(x2, x2) for x2 in xs_2])
                             - 2*gram_matrix(inner)(xs_1, xs_2)))
    return rbf_kernel_sigma_inner

In [ ]:
def poly_kernel_gram(inner, deg, alpha=1.0):
    def poly_kernel_deg_alpha(xs_1, xs_2):
        return (alpha + gram_matrix(inner)(xs_1, xs_2))**deg
    return poly_kernel_deg_alpha

*A quick note on closure: the above kernel construction functions employ a Python concept known as `closure` which provides us the following functionality. Every kernel function should have the same signature: given two data points as input, output a real number. However, the function may be dependent on some parameter that we cannot hard code. Thus, the outer function constructs a kernel function with the desired signature while fixing a value for the parameter. This becomes very useful when we later need to compute kernel values for all pairs of input for a fixed parameter value.*

Let us see how to generate the RBF kernel matrix using `BoW_inner` and `my_inner_product` as $\langle v(x_i), v(x_j)\rangle$ for some value of the parameter $\beta\,.$ Note that `rbf_kernel_gram(inner, beta)` returns a function, and we pass it the datasets as arguments.

In [ ]:
rbf_kernel_gram(BoW_inner,0.2)(X_train[:10],X_train[5:12])

In [ ]:
rbf_kernel_gram(my_inner_product,0.2)(X_train[:10],X_train[5:12])

### Train SVM

Now that we know how to extract and kernelize our data, let us train SVM on it. Note that $C = 1/\lambda\,,$ where $\lambda$ is the regularization parameter we discussed in class. Use a validation set to evaluate the performance of the classifiers you train. As an example, here is how you might use `sklearn`'s SVM implementation:

In [ ]:
svm_clf = svm.SVC(kernel=poly_kernel_gram(BoW_inner, 2))
svm_clf= svm_clf.fit(X_train[:100], y_train[:100])
preds = svm_clf.predict(X_train[200:220])

### [Task 5] Optimizing your SVM classifier
We give you a set of questions below to explore and some direction regarding how to explore them. As your "answer" for this section, submit a write-up in the notebook with 2-3 plots about the answers to these questions. Also submit your code as applicable.

* For differing kernels, train SVM with different $\lambda$ spanning a good range. Use cross validation to determine a good value of $\lambda$. What are the resulted (1) 0-1, Hinge training loss? (2) Margin loss? (3) Test error? (4) Support Vectors?

* Identify examples where the classifier fails for different kernels. Speculate on what the various kernels might be more suited to.

* You implemented your own kernel: how did that do? did the performance match what you were expecting? if not, what factors might have influenced that?

* Consider the various attributes of a machine learning algorithm we may be interested in practically: generalization, runtime, memory usage, ease of implementation, understandability. How does kernel SVM for the kernels you tried perform on each of these attributes?


After evaluating the different classifiers you developed based on various design choices, evaluate the performance of the one you've chosen to "ship". In particular, BEFORE LOADING THE TEST DATA, write your final prediction function, which accepts as input a list of strings to predict on:

In [ ]:
def my_final_predictor(x_data_to_predict_on):
    #### TASK 5 CODE
    my_predictions =      # an array of +1/-1 the same length as x_data_to_predict on
    #### TASK 5 CODE
    return my_predictions

<span style="color: red">
<h4 style="font-weight: bold">[Answer Question 3]</h4>

BEFORE LOADING THE TEST DATA, what is your estimate, based on whatever validation you would like to do using your available training data, for the generalization (ie population) error of my_final_predictor ? <br>

<h4 style="font-weight: bold">---------------------</h4>

<span style="color: blue">
Answer: my_estimated_error_rate = ?????
</span>

<h4 style="font-weight: bold">---------------------</h4>

You are now ready to ship your predictor!!  Hurray!!  
Let's use it, and see how well it does.

In [ ]:
X_test, y_test = utils.load_data(os.path.join(os.getcwd(), "data/cleaned_tweets_test.tsv"))

In [ ]:
test_predictions = my_final_predictor(X_test)
print("My test error:", np.mean(y_test != test_predictions))